<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/Development/District-Builtup-Index-GHSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm.notebook import tqdm

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

[GHSL: Global built-up surface 1975-2030 (P2023A)](https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_BUILT_S)


[ee.ImageCollection.aggregate_array](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-aggregate_array):  Aggregates over a given property of the objects in a collection, calculating a list of all the values of the selected property.

In [2]:
# Load the district boundaries dataset
districts = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM0_NAME', 'India')).select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area'])
districts_dict = districts.select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area']).first().toDictionary().getInfo()
feature = districts.first()

## Load the district geometries

In [3]:
%%time
gdf = geemap.ee_to_gdf(districts.select(['ADM2_CODE', 'ADM2_NAME', 'ADM1_NAME', 'Shape_Area']))

geo_df_simplified = gdf.copy()

## Convert the coordinates to meters
geo_df_simplified = geo_df_simplified.to_crs("EPSG:3857")
## Simplify the geometry by tolerance distance: 1 KM
geo_df_simplified['geometry'] = geo_df_simplified['geometry'].simplify(tolerance=1000,preserve_topology=True)

## Reproject to WGS:84
geo_df_simplified = geo_df_simplified.to_crs("EPSG:4326")

memory_usuage = geo_df_simplified.memory_usage(deep=True).sum()/1024
print(f"Memory usuage of this dataframe is {memory_usuage:.2f} KB")

print(geo_df_simplified.shape)
geo_df_simplified.head()

Memory usuage of this dataframe is 87.48 KB
(573, 5)
CPU times: user 5.15 s, sys: 396 ms, total: 5.55 s
Wall time: 19.6 s


,geometry,ADM1_NAME,ADM2_CODE,ADM2_NAME,Shape_Area
0,"POLYGON ((76.48194 29.51891, 76.49844 29.48948...",Haryana,17660,Karnal,0.240933
1,"POLYGON ((76.4125 30.04788, 76.4283 30.02573, ...",Haryana,17661,Kurukshetra,0.171739
2,"POLYGON ((75.889 28.39892, 75.89713 28.38008, ...",Haryana,17662,Mahendragarh,0.177959
3,"POLYGON ((76.46556 29.24227, 76.50406 29.16966...",Haryana,17665,Sonepat,0.205076
4,"POLYGON ((76.16629 29.92866, 76.18199 29.92324...",Haryana,70134,Kaithal,0.214695


## District wise built up area
Load the image collection containing the built up area

In [4]:
image_collection = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")

years = image_collection.aggregate_array('system:index').getInfo()
print(years)

['1975', '1980', '1985', '1990', '1995', '2000', '2005', '2010', '2015', '2020', '2025', '2030']


- calculate the total builtup area for each district year wise
- export the results

In [ ]:
%%time

# Load the GHSL built-up surface dataset
ghsl_built_up = ee.ImageCollection("JRC/GHSL/P2023A/GHS_BUILT_S")

# Function to calculate the built-up surface area for each district for a given year
def calculate_built_up_area(feature, year):

    district_id = feature.get('ADM2_CODE')

    built_up_image = ghsl_built_up.filterDate(f'{year}-01-01', f'{year}-12-31').first().divide(1e6)

    built_up_area = built_up_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=feature.geometry(),
        scale=100,  # Scale to match GHSL data resolution
        maxPixels=1e9
    ).get('built_surface')

    return feature.set({'district_id':district_id, 'year': year, 'area_km2': built_up_area})

# Initialize an empty list to store results
results_list = []


properties = ['district_id', 'area_km2', 'year']
dfs = []
# Loop through each year and calculate the built-up area for each district
progress_bar1 = tqdm(years, unit="Year", colour='green')
for year in progress_bar1:
    progress_bar1.set_description(f"Processing year: {year}")
    districts_with_built_up_area = districts.map(lambda feature: calculate_built_up_area(feature, year))
    data = {}
    progress_bar2 = tqdm(properties, unit="property", leave=False, colour='blue')
    for prop in progress_bar2:
        progress_bar2.set_description(f"Processing property: {prop}")
        values = districts_with_built_up_area.aggregate_array(prop).getInfo()
        data[prop] = values

    df = pd.DataFrame(data)
    dfs.append(df)

  0%|          | 0/12 [00:00<?, ?Year/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

  0%|          | 0/3 [00:00<?, ?property/s]

 ## Concat the results

In [ ]:
df_final = pd.concat(dfs)
df_final['year'] = df_final['year'].astype(int)
df_final.head()

## Merge the results for one year

In [ ]:
xdf = df_final.loc[(df_final['year']==2005)]
xdf2 = pd.merge(
    left = xdf,
    right = gdf,
    how='left',
    left_on = 'district_id',
    right_on = 'ADM2_CODE'
)
xdf2 = gpd.GeoDataFrame(xdf2)
xdf2.head()

## Visualization

In [ ]:
!pip install -q pydeck

In [ ]:
import base64
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.colors as mp_color
import pydeck as pdk

In [ ]:
def get_color_value(value, cmap, norm):
    # Normalize the value to the range of the colormap
    norm_value = norm(value)

    # Get the color value from the colormap
    color = cmap(norm_value)

    scaled_colors = list(map(lambda x: int(x*255), color[:3]))
    scaled_alpha = int(color[3]*100)

    scaled_colors += [scaled_alpha]
    return scaled_colors


def colormap_dataframe(df, value_col, cmap, norm, max_val=None):

    xdf = df.copy()
    xdf['color'] = xdf[f'{value_col}'].apply(lambda x: get_color_value(x, cmap, norm))
    xdf[['R', 'G', 'B', 'A']] = pd.DataFrame(xdf['color'].to_list())
    xdf = xdf.drop(['color'], axis=1)

    xdf[value_col] = xdf[value_col].round(1)

    return xdf

def generate_log_colorscale(cmap_name, vmin, vmax, size):
    # Generate a range of values from vmin to vmax in log scale
    values = np.logspace(np.log10(vmin), np.log10(vmax), size)

    # Get the colormap
    cmap = plt.get_cmap(cmap_name)

    # Normalize the values to the range [0, 1]
    norm = plt.Normalize(np.log10(vmin), np.log10(vmax))

    # Generate the colorscale
    colors = cmap(norm(np.log10(values)))

    return values, colors

def generate_html_from_fig(image_file):
    # Convert the image to base64 format
    with open(image_file, "rb") as f:
        encoded_image = base64.b64encode(f.read())

    html = "".format(
        encoded_image.decode("utf-8")
    )

    return html

In [ ]:
cmap_name = 'viridis'
vmin=1
vmax=xdf2['area_m2'].max()
size=geo_df_simplified.shape[0]
fontsize = 20

values, colors = generate_log_colorscale(cmap_name, vmin, vmax, size=size)

# Plot the colorscale
plt.figure(figsize=(8, 1))
plt.imshow([colors], aspect='auto', extent=[vmin, vmax, 0, 1])
plt.yticks([])
plt.xticks(fontsize=fontsize)
plt.title(f'Built up area (m2)', fontsize=fontsize)
plt.savefig('legend.jpg',bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
legend_html = generate_html_from_fig("legend.jpg")

In [ ]:
norm = mp_color.LogNorm(vmin, vmax)
geo_df_final = colormap_dataframe(xdf2, value_col='area_m2', cmap=mpl.colormaps[cmap_name], norm=norm)
geo_df_final.sample(5)

In [ ]:
geo_df_json = eval(geo_df_final.to_json())

In [ ]:
INITIAL_VIEW_STATE = pdk.ViewState(latitude=20.5937, longitude=78.9629, zoom=3)

tooltip = {
    "html": """


            State: {ADM1_NAME}<br>
            District: {ADM2_NAME}<br>
            Built-up Area: {area_m2} m²


    """,
    "style": {
        "backgroundColor": "#2c3e50",
        "color": "#ecf0f1",
        "borderRadius": "5px",
        "fontFamily": "Arial, sans-serif",
        "fontSize": "14px",
    }
}

geojson = pdk.Layer(
    type = "GeoJsonLayer",
    data = geo_df_json,
    opacity=0.8,
    pickable=True,
    stroked=True,
    filled=True,
    get_fill_color="[properties.R, properties.G, properties.B, properties.A]",
    get_line_color = [168, 0, 0, 100],
    get_line_width=500,
)

geo_map = pdk.Deck(layers=[geojson], initial_view_state=INITIAL_VIEW_STATE, tooltip=tooltip, description=legend_html)

In [ ]:
geo_map